---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [74]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder





def blight_model():
    

    # Read in the data sets
    
    train = pd.read_csv('train.csv',engine='python')
    test = pd.read_csv('test.csv')
    test1= test
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    
    # Merge the addresses and lat/lons into the train and test DataFrames
    
    addresses = pd.merge(addresses, latlons, how = 'inner')
    test = pd.merge(test, addresses, how = 'inner',left_on='ticket_id', right_on='ticket_id')
    train = pd.merge(train, addresses, how='inner', left_on='ticket_id', right_on='ticket_id')
    
    # dropping all data that exist only in train.csv 
    
    train.drop(['payment_amount', 'payment_date', 'payment_status','balance_due',
               'collection_status', 'compliance_detail','violation_zip_code','discount_amount',
               'violator_name','clean_up_cost','address','mailing_address_str_name','inspector_name',
               'violation_street_name','city','country','violation_description','mailing_address_str_number'
               ], axis=1, inplace=True)
    test.drop(['violation_zip_code','discount_amount','violator_name','clean_up_cost','address',
               'mailing_address_str_name','inspector_name',
              'violation_street_name','city','country','violation_description','mailing_address_str_number'
              ], axis=1, inplace=True)
    
    train.fillna(0, inplace=True)
    test.fillna(0, inplace=True)
    
    train['grafitti_status'] = train['grafitti_status'].replace('GRAFFITI TICKET', 1)
    train['grafitti_status'] = pd.to_numeric(train['grafitti_status'], errors='coerce').fillna(1).astype(int)
    
    test['grafitti_status'] = test['grafitti_status'].replace('GRAFFITI TICKET', 1)
    test['grafitti_status'] = pd.to_numeric(test['grafitti_status'], errors='coerce').fillna(1).astype(int)
    

    # Clear disposition from 'not responsible' and 'pending' values  
    searchfor = ['Not responsible','PENDING']
    train = train[~train.disposition.str.contains('|'.join(searchfor))]

    # Divide 'agency name' and 'dispositio' into onehot encoders
    ctgr1 = train['agency_name']
    ctgr2 = train['disposition']
    
    ctgr1_test = test['agency_name']
    ctgr2_test = test['disposition']
    #labelencoder = LabelEncoder()
    #ctgr = labelencoder.fit(ctgr)
    
    #onehotencoder = OneHotEncoder()
    #oht = onehotencoder.fit_transform(ctgr)#.toarray()
    
    categories1 = pd.get_dummies(ctgr1)
    categories2 = pd.get_dummies(ctgr2)
    
    categories1_test = pd.get_dummies(ctgr1_test)
    categories2_test = pd.get_dummies(ctgr2_test)
    
    final_encoded = pd.DataFrame.join(categories1, categories2).fillna(0)
    final_encoded[final_encoded.columns] = final_encoded[final_encoded.columns].astype(int)
    
    final_encoded_test = pd.DataFrame.join(categories1_test, categories2_test).fillna(0)
    final_encoded_test[final_encoded_test.columns] = final_encoded_test[final_encoded_test.columns].astype(int)
    
    # Proces year and month into int values (using year and month (summer/winter?) as a factor)
    
    year_month_train = train['ticket_issued_date'].str.split('-', expand=True)
    year_month_train.columns =['year', 'month','rest']
    year_month_train = year_month_train.drop(['rest'], axis=1).astype(int)
    
    year_month_test = test['ticket_issued_date'].str.split('-', expand=True)
    year_month_test.columns =['year', 'month','rest']
    year_month_test = year_month_test.drop(['rest'], axis=1).astype(int) 
    
    # Drop get_dummied columns and merge the tables 
    
    train.drop(['agency_name','disposition','ticket_issued_date'], axis=1, inplace=True)
    train = pd.concat([final_encoded, year_month_train, train], axis=1, sort=False)

    test.drop(['agency_name','disposition','ticket_issued_date'], axis=1, inplace=True)
    test = pd.concat([final_encoded_test, year_month_test, test], axis=1, sort=False)
    
    train['Responsible by Determination'] = train['Responsible (Fine Waived) by Deter'
                                                 ] + train['Responsible by Determination']
    train = train.drop(['Responsible (Fine Waived) by Deter'], axis=1)
    
    
    test['Responsible by Admission'] = test['Responsible by Admission'
                                           ] + test['Responsible (Fine Waived) by Admis']
    
    test['Responsible by Determination'] = test['Responsible by Determination'
                                               ] + test['Responsible - Compl/Adj by Determi'
                                                       ] + test['Responsible (Fine Waived) by Deter']
 
    test['Responsible by Default'] = test['Responsible by Default'
                                         ] + test['Responsible - Compl/Adj by Default']
    
    test = test.drop(['Responsible - Compl/Adj by Default',
                      'Responsible (Fine Waived) by Deter',
                      'Responsible - Compl/Adj by Determi','Responsible (Fine Waived) by Admis'
                     ], axis=1,)
    
    train = train.drop(['violation_code','hearing_date','non_us_str_code','zip_code','state'],axis=1)
    test = test.drop(['violation_code','hearing_date','non_us_str_code','zip_code','state'],axis=1)
    

    y = train['compliance'].fillna(0)
    X = train.drop(['compliance'], axis = 1)
    
    # making sure that test and train have the same columns
    new_array = list(set(X.columns) - set(test.columns))
    
    for labels in new_array:
        test[labels] = 0
    test = test.reindex(X.columns.tolist(), axis=1)
    test1=test
    test.fillna('fill')
    # Create X_train data
    #from sklearn.ensemble import RandomForestClassifier
    #regressor = RandomForestClassifier(max_features = 15, random_state = 0)
    #clf = regressor.fit(X,y)
    #rgr = regressor.fit(map(lambda x: [x],train),train_y)
    
     
    #print('Accuracy of RF classifier on training set: {:.2f}'
    #.format(clf.score(X, y)))
    #print('Accuracy of RF classifier on test set: {:.2f}'
    #.format(clf.predict(test)))
    
    return X, y, test
blight_model()

(        Buildings, Safety Engineering & Env Department  \
 0                                                    1   
 1                                                    1   
 5                                                    1   
 6                                                    1   
 7                                                    1   
 8                                                    1   
 9                                                    1   
 12                                                   1   
 13                                                   1   
 14                                                   1   
 18                                                   1   
 19                                                   1   
 21                                                   1   
 22                                                   1   
 23                                                   1   
 26                                                   0 

In [77]:



def trainer():
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    from sklearn.model_selection import GridSearchCV
    
    X_train, y_train, test = blight_model()
    
    grid_values = {'learning_rate': [0.01, 0.1, 1], 'max_depth': [3, 4, 5]}
    clf = GradientBoostingClassifier(random_state = 0)
    grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
    grid_clf_auc.fit(X_train, y_train)
    probs = grid_clf_auc.predict_proba(test)[:, 1]
    result = pd.Series(probs, index=test.ticket_id)
    

    print('Breast cancer dataset')
    print('Accuracy of NN classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
    print('Accuracy of NN classifier on test set: {:.2f}'
     .format(clf.score(test, probs)))
    return 
    
trainer()


Breast cancer dataset


NotFittedError: This GradientBoostingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.